In [3]:
from bs4 import BeautifulSoup
import urllib
import csv


def innerHTML(element):
    return element.decode_contents(formatter="html")

def get_name(body):
	return body.find('span', {'class':'jcn'}).a.string

def get_phone_number(body):
	try:
		return body.find('p', {'class':'contact-info'}).span.a.string
	except AttributeError:
		return ''

def get_rating(body):
	rating = 0.0
	text = body.find('span', {'class':'star_m'})
	if text is not None:
		for item in text:
			rating += float(item['class'][0][1:])/10

	return rating

def get_rating_count(body):
	text = body.find('span', {'class':'rt_count'}).string

	# Get only digits
	rating_count =''.join(i for i in text if i.isdigit())
	return rating_count

def get_address(body):
	return body.find('span', {'class':'mrehover'}).text.strip()

def get_location(body):
	text = body.find('a', {'class':'rsmap'})
	if text == None:
		return
	text_list = text['onclick'].split(",")
	
	latitutde = text_list[3].strip().replace("'", "")
	longitude = text_list[4].strip().replace("'", "")
	
	return latitutde + ", " + longitude

page_number = 1
service_count = 1


fields = ['Name', 'Phone', 'Rating', 'Rating Count', 'Address', 'Location']
out_file = open('Readymade-Garment-Retailers_agra.csv','w')
csvwriter = csv.DictWriter(out_file, delimiter=',', fieldnames=fields)

# Write fields first
#csvwriter.writerow(dict((fn,fn) for fn in fields))

while True:

	# Check if reached end of result
	if page_number > 50:
		break

	url="https://www.justdial.com/Mumbai/Builders-Developers/nct-10059258" % (page_number)
	req = urllib.request.Request(url, headers={'User-Agent' : "Magic Browser"}) 
	page = urllib.request.urlopen( req )
	# page=urllib2.urlopen(url)

	soup = BeautifulSoup(page.read(), "html.parser")
	services = soup.find_all('li', {'class': 'cntanr'})


	# Iterate through the 10 results in the page
	for service_html in services:

		# Parse HTML to fetch data
		dict_service = {}
		name = get_name(service_html)
		phone = get_phone_number(service_html)
		rating = get_rating(service_html)
		count = get_rating_count(service_html)
		address = get_address(service_html)
		location = get_location(service_html)
		if name != None:
			dict_service['Name'] = name
		if phone != None:
			print('getting phone number')
			dict_service['Phone'] = phone
		if rating != None:
			dict_service['Rating'] = rating
		if count != None:
			dict_service['Rating Count'] = count
		if address != None:
			dict_service['Address'] = address
		if location != None:
			dict_service['Address'] = location

		# Write row to CSV
		csvwriter.writerow(dict_service)

		print("#" + str(service_count) + " " , dict_service)
		service_count += 1

	page_number += 1

out_file.close()  

TypeError: not all arguments converted during string formatting

### India Mart

In [3]:
import pandas as pd
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:

PATH = "chromedriver.exe";                               #PATH contains the location where the chromedriver.exe is stored
driver = webdriver.Chrome(PATH)
driver.get("https://dir.indiamart.com/chennai/fresh-vegetables-all.html")       #driver.get() => Contains the URL that needs to be scraped
try:
    content = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "lay-lft"))
    )

    suppliers = content.find_elements_by_class_name("lst_cl")
    # for supplier in suppliers:
    company = [supplier.find_element_by_class_name("gcnm").text for supplier in suppliers]
    address = [supplier.find_element_by_class_name("clg").text for supplier in suppliers]
    precise_add = [supplier.find_element_by_class_name("cty-t").get_attribute("innerText") for supplier in suppliers]
    phone_no = [supplier.find_element_by_css_selector(".bo").get_attribute("innerText") for supplier in suppliers]
    items_list = []
    for supplier in suppliers:
        # items = supplier.find_elements_by_class_name("cp5")
        item = [item.find_element_by_class_name("gpnm").text for item in supplier.find_elements_by_class_name("cp5")]
        items_list.append(item)

finally:
    driver.quit()

supplier_det = pd.DataFrame({
    'Company': company,
    'Address': address,
    'Precise_add': precise_add,
    'Phone_no': phone_no,
    'Items': items_list
})

print(supplier_det)
#supplier_det.to_csv("sup_det.csv")

                            Company                                Address  \
0               Hipster Enterprises           Venkateshwara Nagar, Chennai   
1                     Afrah Trading              Old Washermanpet, Chennai   
2                 Prime India Impex                    Manapakkam, Chennai   
3                       Zia Exports                        Pattur, Chennai   
4                    United Exports                 Virugambakkam, Chennai   
5            Goodwill Exports India                 West Mambalam, Chennai   
6                      Hi Fi Trades                     Mannurpet, Chennai   
7                  Karthick Traders                                Chennai   
8   M/s. Pee Pee Appliances Pvt Ltd                    Ayanavaram, Chennai   
9                     Suriya Greens                                Chennai   
10          Grand Depatmental Store                      Saidapet, Chennai   
11                      VML Exports                             

In [22]:

PATH = "chromedriver.exe";                               #PATH contains the location where the chromedriver.exe is stored
driver = webdriver.Chrome(PATH)
driver.get("https://dir.indiamart.com/chennai/contractor-contractors.html")  


storeDetails = driver.find_elements_by_class_name('clg')
names = driver.find_elements_by_class_name('lcname')
contactList = driver.find_elements_by_class_name('pns_h')

nameList = []
addressList = []
numbersList = []

for i in range(len(names)):
    
    name = names[i].text
    address = storeDetails[i].text
    contact = contactList[i].get_attribute("innerHTML")
    
    nameList.append(name)
    addressList.append(address)
    numbersList.append(contact)


    
# intialise data of lists.
data = {'Company Name':nameList,
        'Address': addressList,
        'Phone':numbersList}
#Make DataFrame
df = pd.DataFrame(data)

In [31]:
PATH = "chromedriver.exe";                               #PATH contains the location where the chromedriver.exe is stored
driver = webdriver.Chrome(PATH)
driver.get("https://dir.indiamart.com/chennai/plumbing-contractors.html")  


storeDetails = driver.find_elements_by_class_name('clg')
names = driver.find_elements_by_class_name('lcname')
contactList = driver.find_elements_by_class_name('pns_h')

nameList = []
addressList = []
numbersList = []

for i in range(len(names)):
    
    name = names[i].text
    address = storeDetails[i].text
    contact = contactList[i].get_attribute("innerHTML")
    
    nameList.append(name)
    addressList.append(address)
    numbersList.append(contact)


    
# intialise data of lists.
data = {'Company Name':nameList,
        'Address': addressList,
        'Phone':numbersList}

# Create DataFrame
df = pd.DataFrame(data)

In [32]:
df

,Company Name,Address,Phone
0,Synergy Automatics,"Ambattur, Chennai",08047007911
1,Samprid Infra,"Urapakkam, Chennai",08046035229
2,Saif Ceramics,"George Town, Chennai",08048412681
3,Sri Hari Ventilators,"Madhavaram, Chennai",08048974889
4,Dynamic Cutting Technologies,"Valasaravakkam, Chennai",08042964378
5,Straits Engineering Private Limited,Chennai,08048929330
6,EL Engineering & Fabrication,"Kovilambakkam, Chennai",08048797585
7,Apex Sanitech Systems,"Nangainallur, Chennai",08048737854
8,Lakshmi Corporations,"George Town, Chennai",08048763522
9,Aamphaa Projects,Chennai,08048710226


In [40]:
PATH = "chromedriver.exe";                               #PATH contains the location where the chromedriver.exe is stored
driver = webdriver.Chrome(PATH)
driver.get("https://dir.indiamart.com/search.mp?ss=plumbing+system+%26+sanitary+system&src=as-default%3Apos%3D1%3Acat%3D260%3Amcat%3D175155&prdsrc=1&mcatid=175155&catid=260&cq=noida&cq_src=mcat")  


storeDetails = driver.find_elements_by_class_name('clg')
names = driver.find_elements_by_class_name('lcname')
contactList = driver.find_elements_by_class_name('pns_h')

nameList = []
addressList = []
numbersList = []

for i in range(len(names)):
    
    name = names[i].text
    address = storeDetails[i].text
    contact = contactList[i].get_attribute("innerHTML")
    
    nameList.append(name)
    addressList.append(address)
    numbersList.append(contact)


    
# intialise data of lists.
data = {'Company Name':nameList,
        'Address': addressList,
        'Phone':numbersList}

# Create DataFrame
df = pd.DataFrame(data)
df

,Company Name,Address,Phone
0,GENERAL CRAFT SOLUTIONS,"Block A sector 16, Noida",08048928011
1,Material Movell India Private Limited,"Sector 63, Noida",08048081169
2,Vamani Overseas Pvt Ltd,"Ambedkar Vihar, Noida",09716383068
3,E P I Construction,Noida,08048782114
4,Kalco Infratech Private Limited,Deals in Noida,07971385161
5,Mechno Tech Facility Management Services Pvt Ltd,13 km from Noida,"08048371759 <span class=""fs15 mFwn..."
6,Delta Aircon,Deals in Noida,08048978707
7,M/s Hk Sanitation,13 km from Noida,08048846661
8,Navesha Corporation.,Deals in Noida,07971376909
9,Adrshan Refrigeration & Electrical Solutions,Deals in Noida,08046074187


In [37]:
df.to_excel('saved_file.xlsx')